<a href="https://colab.research.google.com/github/mailaucq/book_classification/blob/main/roberta_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import transformers
from transformers import BertTokenizer, BertForSequenceClassification 
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW

import warnings
warnings.filterwarnings("ignore")


print(torch.__version__)


1.6.0a0+bf2bbd9


In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_TYPE = 'xlm-roberta-base'

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)

In [ ]:
tokenizer.vocab_size

250002

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

In [ ]:
## Mount Drive into Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = "/content/drive/My Drive/Colab Notebooks/projeto-reconhecimento-autoria/dataset/"

In [ ]:
name_dataset = "dataset_2"
length_cut = 1000
random_flag = False

In [ ]:
df_train = pd.read_csv(dataset_path + "df_train_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [ ]:
df_val = pd.read_csv(dataset_path + "df_val_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [ ]:
df_test = pd.read_csv(dataset_path + "df_test_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [ ]:
df_val.shape

(8300, 2)

In [ ]:
df_val.head()

,text,label
0,GEOLOGICAL OBSERVATIONS ON SOUTH AMERICA by CH...,charlesDarwin
1,present volume and both the general reader who...,charlesDarwin
2,OR USPALLATA PASS SECTION SECTION OF THE VALLE...,charlesDarwin
3,of the elevatory process Mode of formation of ...,charlesDarwin
4,Upraised shells of Cobija Iquique and Arica Li...,charlesDarwin


In [ ]:
MODEL_TYPE = 'xlm-roberta-base'


L_RATE = 1e-6 #1e
MAX_LEN = 256 #256

NUM_EPOCHS = 10
BATCH_SIZE = 32 #32
NUM_CORES = os.cpu_count()

NUM_CORES

2

In [ ]:
label_to_ix = {}
for label in df_train.label:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)
label_to_ix

{'allanPoe': 16,
 'andrewLang': 3,
 'arthurDoyle': 18,
 'bmBower': 2,
 'bramStoker': 11,
 'charlesDarwin': 4,
 'charlesDickens': 17,
 'hectorMunro': 19,
 'henryJames': 10,
 'hermanMelville': 0,
 'hgWells': 15,
 'horatioAlger': 12,
 'janeAusten': 7,
 'markTwain': 6,
 'nathanielHawthorne': 1,
 'pgWodehouse': 9,
 'richardHarding': 14,
 'thomasHardy': 5,
 'whashingtonIrving': 8,
 'zaneGrey': 13}

In [ ]:
total_classes = list(set(df_train['label']))
len(total_classes)

20

In [ ]:
number_books = (df_train[df_train['label'] == total_classes[0]]).shape[0]
number_books

1245

In [ ]:
!pip install scikit-multilearn==0.2.0

In [ ]:
df_train.label.value_counts()

richardHarding        1245
hgWells               1245
markTwain             1245
pgWodehouse           1245
janeAusten            1245
arthurDoyle           1245
hermanMelville        1245
whashingtonIrving     1245
henryJames            1245
bramStoker            1245
andrewLang            1245
allanPoe              1245
charlesDickens        1245
thomasHardy           1245
nathanielHawthorne    1245
zaneGrey              1245
charlesDarwin         1245
hectorMunro           1245
bmBower               1245
horatioAlger          1245
Name: label, dtype: int64

In [ ]:
df_val.label.value_counts()

richardHarding        415
hermanMelville        415
horatioAlger          415
charlesDarwin         415
whashingtonIrving     415
hectorMunro           415
henryJames            415
markTwain             415
pgWodehouse           415
allanPoe              415
bmBower               415
charlesDickens        415
janeAusten            415
arthurDoyle           415
bramStoker            415
hgWells               415
thomasHardy           415
nathanielHawthorne    415
zaneGrey              415
andrewLang            415
Name: label, dtype: int64

In [ ]:
df_test.label.value_counts()

richardHarding        415
hermanMelville        415
horatioAlger          415
charlesDarwin         415
whashingtonIrving     415
hectorMunro           415
henryJames            415
markTwain             415
pgWodehouse           415
allanPoe              415
bmBower               415
charlesDickens        415
janeAusten            415
arthurDoyle           415
bramStoker            415
hgWells               415
thomasHardy           415
nathanielHawthorne    415
zaneGrey              415
andrewLang            415
Name: label, dtype: int64

In [ ]:
device = xm.xla_device()

print(device)

xla:1


In [ ]:
df_train.head()

,text,label
0,THE PIAZZA TALES by HERMAN MELVILLE Author of ...,hermanMelville
1,the freedom of out doors and it is so pleasant...,hermanMelville
2,been The house is old Seventy years since from...,hermanMelville
3,better than he knew or else Orion in the zenit...,hermanMelville
4,much of an omission as if a picture gallery sh...,hermanMelville


In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

# xlm-roberta-large
print('Loading XLMRoberta tokenizer...')
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)

Loading XLMRoberta tokenizer...


In [ ]:
class BookDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'text']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1,           # Sentences to encode.
                    add_special_tokens = True,      # Add the special tokens.
                    max_length = MAX_LEN,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # Convert the target to a torch tensor
        target = torch.tensor(label_to_ix[self.df_data.loc[index, 'label']])

        sample = (padded_token_list, att_mask, target)


        return sample


    def __len__(self):
        return len(self.df_data)

In [ ]:
from transformers import XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = len(list(label_to_ix.values())), # The number of output labels. 2 for binary classification.
)

# Send the model to the device.
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [ ]:
# Define the optimizer
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

In [ ]:
# Create the dataloaders.

train_data = BookDataset(df_train)
val_data = BookDataset(df_val)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)




print(len(train_dataloader))
print(len(val_dataloader))


779
260


In [ ]:
os.environ['XLA_USE_32BIT_LONG'] = '1'

In [ ]:
%%time


# Set the seed.
seed_val = 101

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []


# For each epoch...
for epoch in range(0, NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    

    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # put the model into train mode
    model.train()
    
    # This turns gradient calculations on and off.
    torch.set_grad_enabled(True)


    # Reset the total loss for this epoch.
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        


        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_train_loss = total_train_loss + loss.item()
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        
        
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        
        # Use the optimizer to update the weights.
        
        # Optimizer for GPU
        # optimizer.step() 
        
        # Optimizer for TPU
        # https://pytorch.org/xla/
        xm.optimizer_step(optimizer, barrier=True)

    
    print('Train loss:' ,total_train_loss)


    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # Put the model in evaluation mode.
    model.eval()

    # Turn off the gradient calculations.
    # This tells the model not to compute or store gradients.
    # This step saves memory and speeds up validation.
    torch.set_grad_enabled(False)
    
    
    # Reset the total loss for this epoch.
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_val_loss = total_val_loss + loss.item()
        

        # Get the preds
        preds = outputs[1]


        # Move preds to the CPU
        val_preds = preds.detach().cpu().numpy()
        
        # Move the labels to the cpu
        targets_np = b_labels.to('cpu').numpy()

        # Append the labels to a numpy list
        targets_list.extend(targets_np)

        if j == 0:  # first batch
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

    
    # Calculate the validation accuracy
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)


    # Save the Model
    torch.save(model.state_dict(), 'model.pt')
    
    # Use the garbage collector to save memory.
    gc.collect()


======== Epoch 1 / 10 ========
Training...
Train loss: 2231.252442598343

Validation...
Val loss: 692.3582408428192
Val acc:  0.28373493975903613

======== Epoch 2 / 10 ========
Training...
Train loss: 1883.1287261247635

Validation...
Val loss: 602.8517423868179
Val acc:  0.3680722891566265

======== Epoch 3 / 10 ========
Training...
Train loss: 1622.9499034881592

Validation...
Val loss: 546.1748535633087
Val acc:  0.41096385542168673

======== Epoch 4 / 10 ========
Training...
Train loss: 1414.0574227571487

Validation...
Val loss: 514.4173811674118
Val acc:  0.4566265060240964

======== Epoch 5 / 10 ========
Training...
Train loss: 1229.0786238908768

Validation...
Val loss: 481.63317704200745
Val acc:  0.48168674698795183

======== Epoch 6 / 10 ========
Training...
Train loss: 1058.8732355833054

Validation...
Val loss: 465.34632086753845
Val acc:  0.4834939759036145

======== Epoch 7 / 10 ========
Training...
Train loss: 908.5823633670807

Validation...
Val loss: 451.18812799453